In [ ]:
import nltk
from nltk.corpus import brown, stopwords
from nltk import RegexpTokenizer, FreqDist, ngrams
import numpy as np
import pandas as pd
import re, pickle, random, math

In [ ]:
def make_words_grams():
# print(list(brown.words()[:500]))
    words = [w.lower() for w in brown.words()]
    # words = [w for w in words if w not in stopwords.words('english')]
    estopwords = set(stopwords.words('english'))
    words = [w for w in words if w not in estopwords]
    rem = re.compile(r'[a-z][a-z\']*')
    words = [w for w in words if rem.search(w)]
    
    freq_words = FreqDist(words).most_common(5000)
    freq_words = set([w for (w,f) in freq_words])
    words = [w for w in words if w in freq_words]
    
    grams = list(ngrams(words, 15))
    pickle.dump(words, open("words.txt", "wb"))
    pickle.dump(grams, open("grams.txt", "wb"))
    return words, grams

In [ ]:
## load words and grams
with open("words.txt", "rb") as fp:
    words = pickle.load(fp)
with open("grams.txt", "rb") as fp:
    grams = pickle.load(fp)

In [ ]:
words, grams = make_words_grams()

In [ ]:
vocab = list(set(words))
voc_len = len(vocab)
matrix = pd.DataFrame(0, vocab, vocab, dtype=np.float32)
# print(matrix)

In [ ]:
print(voc_len, matrix.shape, len(grams))


In [ ]:
for g in grams:
    mid_word = g[7]
#     print(mid_word)
    row = matrix.loc[mid_word]
    row.at[g[0]]  += 1
    row.at[g[1]]  += 1
    row.at[g[2]]  += 1
    row.at[g[3]]  += 1
    row.at[g[4]]  += 1
    row.at[g[5]]  += 1
    row.at[g[6]]  += 1
    row.at[g[8]]  += 1
    row.at[g[9]]  += 1
    row.at[g[10]] += 1
    row.at[g[11]] += 1
    row.at[g[12]] += 1
    row.at[g[13]] += 1
    row.at[g[14]] += 1

#     for i in range(len(g)):
#         if i == 7:
#             continue
#         row.at[g[i]] += 1
#         print(i, g[i], row.at[g[i]])


In [ ]:
xy = matrix.loc["play"].iloc[matrix.loc["play"].nonzero()]
print(xy.sort_values(ascending=False)[:10])


In [ ]:
matrix.shape

In [ ]:
matrix.to_hdf("matrix5000.hdf","mat1", mode='w')



In [ ]:
matrix = pd.read_hdf('matrix5000.hdf','mat1')

In [ ]:
cols = matrix.sum(axis=1)
rows = matrix.sum(axis=0)

In [ ]:
# log_2 val / (row * col)
for r in matrix.index:
    row = matrix.loc[r]
    for c in matrix.columns:
        val = row.at[c]
        if val <= 0: 
            row.at[c] = 0
            continue
            
        ppmi = row.at[c] / (cols[c] * float(rows[r]))
        ppmi = max(0, math.log2(ppmi))
        row.at[c] = ppmi

In [ ]:
matrix.columns

In [ ]:
matrix.to_hdf("ppmi_matrix.hdf","mat2", mode='w')
